In [11]:
import socket
import socketserver, time

In [22]:
host = "25.114.157.253"  #http://25.94.218.230:555
port = 1234
timeout = 5
buffer_size = 1024

aux = 0


In [ ]:
server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
server.settimeout(timeout)
server.bind((host, port))

print('Udp conectado')

In [ ]:
def receive_msg():
    
    rcv_bytes = server.recvfrom(buffer_size)
    message = rcv_bytes[0]
    address = rcv_bytes[1]
    
    print('message: ', message.decode())
    print('from: ', address)
    
    confirmation = 'Message received!'
    
    server.sendto(confirmation, address)

In [ ]:
def send_msg():
    pass

In [ ]:
# alive = True

# while alive:
    
#     frame = UDP